<a href="https://colab.research.google.com/github/Kaushikpandav/LipsSync/blob/main/Wav2LipAntriksh_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to my Easy Wav2Lip colab!

My goal is to make lipsyncing with this tool easy, fast and great looking!

Please view the GitHub for instructions: [https://github.com/vidhanmehta/lipsync](https://github.com/vidhanmehta/lipsync?tab=readme-ov-file#best-practices)

In [2]:
version = 'v8.3'
#@title <h1>Step 1: Setup "Easy-Wav2Lip"</h1> With one button: it's really that easy!
#@markdown 👈 Click that little circle play button first - it will ask for Google Drive access: <br>
#@markdown > Accept if your files are on Google Drive (recommended).
#@markdown <br> Alternatively, you can click deny and upload files manually, but this is slower.

#check if already installed
import os
import sys
if os.path.exists('installed.txt'):
  with open('last_file.txt', 'r') as file:
    last_file = file.readline()
  if last_file == version:
    sys.exit('Easy-Wav2Lip '+version+' has already been run on this instance!')


#check GPU is enabled
print('checking for GPU')
import torch
if not torch.cuda.is_available():
  sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')

#prompt to mount google drive
print('requesting Google Drive access')
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("google drive not linked")


#start timer
import time
start_time = time.time()

#clone git
giturl = 'https://github.com/anothermartz/Easy-Wav2Lip.git'


!git clone -b {version} {giturl}
%cd 'Easy-Wav2Lip'
working_directory = os.getcwd()
!mkdir 'face_alignment' 'temp'

#install prerequisites
print('installing batch_face')
import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        module='torchvision.transforms.functional_tensor')
!pip install batch_face --quiet
!pip install basicsr==1.4.2 --quiet
print('fixing basicsr degradations.py')
!cp /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py
print('installing gfpgan')
!pip install gfpgan --quiet

!python install.py

from IPython.display import clear_output
clear_output()
print("Installation complete, move to Step 2!")

#end timer
elapsed_time = time.time() - start_time
from easy_functions import format_time
print(f"Execution time: {format_time(elapsed_time)}")

Installation complete, move to Step 2!
Execution time: 47s


In [4]:
import os
import sys
import time
import configparser
from easy_functions import show_video
from IPython.display import Image

# Step 1 Check
if not os.path.exists('installed.txt'):
    sys.exit('Step 1 has not been run in this instance! Please run step 1 each time you disconnect from a runtime.')

############################## user inputs #####################################
# Step 2: Select inputs
# On desktop: Click the folder icon ( 📁 ) at the left edge of colab, find your file, right click, copy path, paste it below:
video_file = "/content/vvvv.mp4"  # @param {type:"string"}
vocal_file = "/content/vvv2.wav"  # @param {type:"string"}

# Quality
quality = "Enhanced"  # @param ["Fast", "Improved", "Enhanced"]
output_height = "full resolution"  # @param ["half resolution", "full resolution", "480"] {allow-input: true}
use_previous_tracking_data = True  # @param {type:"boolean"}

# Step 3: Click the little circle play button on this cell! (Or press ctrl + F10) - Then wait for processing to complete.
# Scale padding with resolution
wav2lip_version = "Wav2Lip"  # @param ["Wav2Lip", "Wav2Lip_GAN"]
nosmooth = True  # @param {type:"boolean"}

# Padding (Up, Down, Left, Right)
U = 0  # @param {type:"slider", min:-100, max:100, step:1}
D = 10  # @param {type:"slider", min:-100, max:100, step:1}
L = 0  # @param {type:"slider", min:-100, max:100, step:1}
R = 0  # @param {type:"slider", min:-100, max:100, step:1}

# Mask
size = 1.5  # @param {type:"slider", min:1, max:6, step:0.1}
feathering = 1  # @param {type:"slider", min:0, max:3, step:1}
mouth_tracking = False  # @param {type:"boolean"}
debug_mask = False  # @param {type:"boolean"}

# Other options
batch_process = False  # @param {type:"boolean"}
output_suffix = "_Easy-Wav2Lip"  # @param {type:"string"}
include_settings_in_suffix = False  # @param {type:"boolean"}
preview_input = False  # @param {type:"boolean"}
preview_settings = False  # @param {type:"boolean"}
frame_to_preview = 100  # @param {type:"integer"}

# Create a ConfigParser object
config = configparser.ConfigParser()

# Put all your variables in a dictionary
options = {
    'video_file': video_file,
    'vocal_file': vocal_file,
    'quality': quality,
    'output_height': output_height,
    'wav2lip_version': wav2lip_version,
    'use_previous_tracking_data': use_previous_tracking_data,
    'nosmooth': nosmooth
}
padding = {
    'U': U,
    'D': D,
    'L': L,
    'R': R
}
mask = {
    'size': size,
    'feathering': feathering,
    'mouth_tracking': mouth_tracking,
    'debug_mask': debug_mask
}
other = {
    'batch_process': batch_process,
    'output_suffix': output_suffix,
    'include_settings_in_suffix': include_settings_in_suffix,
    'preview_input': preview_input,
    'preview_settings': preview_settings,
    'frame_to_preview': frame_to_preview
}

# Add the dictionary to the ConfigParser object
config['OPTIONS'] = options
config['PADDING'] = padding
config['MASK'] = mask
config['OTHER'] = other

# Write the data to an INI file
with open('config.ini', 'w') as f:
    config.write(f)

# Show the default video only the first time the notebook is opened
default_video_displayed = False

# Check if the user is running the notebook for the first time
if not os.path.exists('default_video_shown.txt'):
    default_video_displayed = True
    # Change this to your desired default video file path
    default_video_file = "/content/output (11).mp4"  # Replace with your default video file path

    # Check if the default video file exists before displaying it
    if os.path.isfile(default_video_file):
        print(f"Loading default video preview...")
        show_video(default_video_file)  # Display the default video
    else:
        print(f"Default video file '{default_video_file}' does not exist!")

    # Create a file to indicate the default video has been shown
    with open('default_video_shown.txt', 'w') as f:
        f.write("Default video displayed.")

# Run the Wav2Lip processing
!python run.py

# Display preview settings or output video based on conditions
if preview_settings:
    if os.path.isfile(os.path.join('temp', 'preview.jpg')):
        display(Image(os.path.join('temp', 'preview.jpg')))
else:
    # Check if the output video file exists before displaying it
    if os.path.isfile(os.path.join('temp', 'output.mp4')):
        print(f"Loading processed video preview...")
        show_video(os.path.join('temp', 'output.mp4'))


Processing vvvv.mp4 using vvv2.wav for audio
imports loaded!     
analysing audio...
295 frames to process
detecting face in every frame: 100%|██████████████████████████████| 295/295 [00:05<00:00, 57.43it/s]
mask size: 1.5, feathering: 1
Loading gfpgan
Starting...
Processing Wav2Lip: 100%|█████████████████████████████████████████| 295/295 [01:43<00:00,  2.85it/s]
converting to final video
vvvv_vvv2 successfully lip synced! It will be found here:
/content/vvvv_vvv2_Easy-Wav2Lip.mp4
Execution time: 2m 12s
Loading processed video preview...
